In [2]:
import os
# Print the current working directory
cwd = os.getcwd()
if cwd.endswith('SOM_package'):
    print('Current working directory is already set to the package directory')
else:
    os.chdir(cwd + '/SOM_package')
from som_data_struct import som_data_struct
from som_normalize import som_normalize
from som_make import som_make
from som_bmus import som_bmus
from som_ind2sub import som_ind2sub
from som_denormalize import som_denormalize
from reader import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import copy
from sklearn.cluster import KMeans
os.chdir('../')


In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

def get_start_end_indices(labels_dataframe, ID):
        start_test = labels_dataframe.groupby('Participant').head(1).index.tolist()
        end_test = start_test[1:] + [len(labels_dataframe)]
        return start_test, end_test

def correct_end_position(end_test):
    end_test_c = [x-1 for x in end_test]
    end_test_c[-1] =end_test_c[-1]+1
    return end_test_c

# Read the data
### Placeholder datafunction
data_path = ['ME.data', 'LC.data', 'Controls.data']
for data_p in data_path:
    data_set = pd.read_csv(data_p, delimiter=' ', na_values='NaN', header=None, skiprows=2)
    
    # Extract headers
    header_start = data_set.shape[1] - 2
    headers = read_headers(data_p)
    headers = [header.strip() for header in headers]
    
    # Define labels and component names
    label_names = headers[header_start:]
    compnames = headers[0:header_start]
    compnames = np.array(compnames)
    
    # Extract labels and data
    labels = data_set.iloc[:, header_start:]
    labels = labels.to_numpy()
    data = data_set.iloc[:, 0:header_start]
    data = data.to_numpy()
    
    # Create the structured data dictionary
    sData = som_data_struct(data.copy())
    sData['label_names'] = label_names
    sData['comp_names'] = compnames
    sData['labels'] = labels
    sData_copy = copy.deepcopy(sData)
    dataframe_labels = pd.DataFrame(sData['labels'])
    dataframe_labels.columns = sData['label_names']

    # Remove Phase NONE and get start and end indices
    ID = dataframe_labels.iloc[:,0].unique()
    phase = dataframe_labels.iloc[:,1].unique()
    start_test, end_test = get_start_end_indices(dataframe_labels, ID)
    temp_labels = dataframe_labels[dataframe_labels['Phase'] != 'NONE']
    sData['labels'] = temp_labels.to_numpy()
    # MAKING BREATH TIMES
    MIN = []
    MAX = []
    breath_time = []
    sData_data_df = pd.DataFrame(sData['data'])
    sData_data_df.columns = sData['comp_names']
    for i in range(len(ID)):
        start = start_test[i]
        end = end_test[i]
        bt = np.diff(sData_data_df.iloc[start:end, 0].to_numpy(), prepend=0)
        #bt[bt < -100] = np.nan
        breath_time.append(bt)
        MIN.append(np.nanmin(bt))
        MAX.append(np.nanmax(bt))        
    breath_data = sData_data_df.iloc[:,0]
    sData_data_df.iloc[:,0] = pd.Series(np.concatenate(breath_time))
    end_test_c = correct_end_position(end_test) 
    # FIND TRIAL PHASES FOR EACH PARTICIPANT
    start_exercise = []
    end_exercise = []
    for i in range(len(ID)):
        ip = temp_labels[start_test[i]:end_test_c[i]].groupby('Phase').head(1)
        start_exercise.append(ip[ip['Phase'] == 'EXERCISE'].index[0])
        end_exercise.append((temp_labels[start_test[i]:end_test_c[i]]['Phase'] == 'EXERCISE').index[-1])
    
    # CREATE A DICT FOR EXERCISE PART OF TRIALS
    exercise_cell = [sData_data_df.iloc[start_exercise[i]:end_exercise[i]].to_numpy() for i in range(len(ID))]
    labels_exercise = [np.repeat(ID[i], len(exercise_cell[i])) for i in range(len(ID))]
    allLabels = np.concatenate(labels_exercise)

    sData_exercise = som_data_struct(np.vstack(exercise_cell))
    sData_exercise['label_names'] = [label_names[0]]
    sData_exercise['comp_names'] = compnames
    sData_exercise['labels'] = allLabels 
    dataframe_labels_exercise = pd.DataFrame(sData_exercise['labels'])
    dataframe_labels_exercise.columns = [sData_exercise['label_names'][0]]
    start_test_exercise, end_test_exercise = get_start_end_indices(dataframe_labels_exercise, ID)
    breath_nr = np.concatenate([np.arange(1, end_test_exercise[i] - start_test_exercise[i] + 2) for i in range(len(ID))])


    if data_p == 'ME.data':
        sData_ME = sData_exercise.copy()
        sData_ME_copy = copy.deepcopy(sData_ME)
        plotdata_ME = sData_ME['data'].copy
        sData_ME_norm = som_normalize(sData_ME, 'var')
        sData_ME_norm_copy = copy.deepcopy(sData_ME_norm)
    elif data_p == 'LC.data':
        sData_LC = sData_exercise.copy()
        sData_LC_copy = copy.deepcopy(sData_LC)
        plotdata_LC = sData_LC['data'].copy
        sData_LC_norm = som_normalize(sData_LC, 'var')
        sData_LC_norm_copy = copy.deepcopy(sData_LC_norm)
    elif data_p == 'Controls.data':
        sData_controls = sData_exercise.copy()
        sData_controls_copy = copy.deepcopy(sData_controls)
        plotdata_controls = sData_controls['data'].copy
        sData_controls_norm = som_normalize(sData_controls, 'var')
        sData_controls_norm_copy = copy.deepcopy(sData_controls_norm)
    else:
        print('Error in data reading: Weird data path')
#sMap = som_make(sData_controls_norm, *['lattice', 'shape', 'training'],**{'lattice':'hexa', 'shape':'sheet', 'training': 'long'})  
#sMap['comp_names'] = headers


In [15]:
sData_data_df

,t,Power,HR,VE,VO2,VCO2,PetCO2,PetO2,VO2/Kg,VE/VO2,VE/VCO2,RQ,VT,Rf,Ti,Te
0,5.0,0.0,NaN,8.062,324.304065,234.075281,33.0,108.0,4.36,22.2,30.7,0.72,0.653,12.35,1.87,2.99
1,7.0,0.0,NaN,7.640,342.544765,239.146688,35.0,103.0,4.61,20.4,29.3,0.70,0.834,9.16,1.96,4.59
2,3.0,0.0,NaN,8.733,368.639749,255.788145,32.0,108.0,4.96,20.9,30.1,0.69,0.588,14.85,1.80,2.24
3,5.0,0.0,NaN,8.354,356.479196,249.670125,32.0,107.0,4.80,20.6,29.4,0.70,0.582,14.35,1.51,2.67
4,5.0,0.0,NaN,7.964,351.126102,247.138020,34.0,105.0,4.73,20.3,28.9,0.70,0.673,11.83,2.05,3.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9636,3.0,360.0,122.0,45.829,1079.996562,1205.360978,29.0,121.0,14.34,41.4,37.1,1.12,2.765,16.57,1.15,2.47
9637,5.0,365.0,122.0,22.694,498.584751,542.316133,32.0,115.0,6.62,43.9,40.4,1.09,1.963,11.56,1.24,3.95
9638,3.0,365.0,124.0,60.412,1567.376216,1623.980397,30.0,118.0,20.82,37.6,36.3,1.04,2.930,20.62,1.21,1.70
9639,3.0,365.0,122.0,55.416,1301.385957,1398.073963,28.0,120.0,17.28,41.5,38.7,1.07,2.817,19.67,1.18,1.87


In [130]:
pd.DataFrame(sData_ME_norm['data']).isna().sum()

0      0
1      0
2     61
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12     0
13     0
14     0
15     0
dtype: int64

Construct breath number index

In [ ]:
    # CHANGES IN WATTS
    # individualize
    data_exercise = pd.DataFrame(sData_exercise['data'])
    dif_data = [np.diff(data_exercise.iloc[start_test_exercise[i]:end_test_exercise[i], 1]) for i in range(len(ID))]
    
    # NORMALIZE DATA
    differences = []
    max_diff = 0
    trial_length1 = dataframe_labels_exercise.groupby('Participant').head(1).index.tolist()
    for i in range(len(trial_length1)):
        if i < len(trial_length1)-1:
            diff = trial_length1[i+1] - trial_length1[i]
            differences.append(diff)
        if i == len(trial_length1)-1:
            diff = len(dataframe_labels_exercise) - trial_length1[i]
            differences.append(diff)
        if diff > max_diff:
            max_diff = diff
            max_diff_index = i

    #Normalised based on Duration of trial, Because starting resistance differed between trials  
    diff_normalised = [max_diff/difference for difference in differences]
    data_normalized = [data_exercise.iloc[start_test_exercise[i]:end_test_exercise[i]].apply(lambda x: x * diff_normalised[i], axis=1).to_numpy() for i in range(len(ID))]
    data_cell = []
    for i in range(len(differences)):
        trial_matrix = np.full((max_diff, data_exercise.shape[1]), np.nan)
        data_cell.append(0)
        trial_matrix[:differences[i], :data_exercise.shape[1]] = data_normalized[i]
        data_cell[i] = trial_matrix
    data_cell_v = np.vstack(data_cell)
    data_cell_df = pd.DataFrame(data_cell_v)
    data_cell_df.columns = sData_exercise['comp_names']
    sData_normalized = som_data_struct(data_cell_v)
    sData_normalized['label_names'] = [label_names[0]]
    sData_normalized['comp_names'] = compnames
    sData_normalized['labels'] = allLabels

In [102]:
WATTS = data_cell_df['Power']
data_cell_df.drop(columns=['t', 'Power'], inplace=True)
if data_p == 'LC.data':
    

SEPARATE TARGET VARIABLE FROM DATA

In [11]:
if data_path == 'LC.data':
    datapoints_LC = len(data_normalized['labels'])
    LC_data = data_normalized['data'].copy()
    LC_labels = data_normalized['labels'].copy()
    LC_dict = {"type":data_normalized["type"], "data": LC_data, "labels": LC_labels, "comp_names": data_normalized["comp_names"], "comp_norm": data_normalized["comp_norm"], "label_names": data_normalized["label_names"]}
    LC_dict['labels'] = pd.DataFrame(LC_dict['labels'], columns = ['participant'])
    LC_dict['data'].columns = compnames[2:]

if data_path == 'Controls.data':
    datapoints_controls = len(data_normalized['labels'])
    Control_data = data_normalized['data'].copy()
    Control_labels = data_normalized['labels'].copy()
    Control_dict = {"type":data_normalized["type"], "data": Control_data, "labels": Control_labels, "comp_names": data_normalized["comp_names"], "comp_norm": data_normalized["comp_norm"], "label_names": data_normalized["label_names"]}
    Control_dict['labels'] = pd.DataFrame(Control_dict['labels'], columns = ['participant'])
    Control_dict['data'].columns = compnames[2:]

if data_path == 'ME.data':
    datapoints_me = len(data_normalized['labels'])
    ME_data = data_normalized['data'].copy()
    ME_labels = data_normalized['labels'].copy()
    ME_dict = {"type":data_normalized["type"], "data": ME_data, "labels": ME_labels, "comp_names": data_normalized["comp_names"], "comp_norm": data_normalized["comp_norm"], "label_names": data_normalized["label_names"]}
    ME_dict['labels'] = pd.DataFrame(ME_dict['labels'], columns = ['participant'])
    ME_dict['data'].columns = compnames[2:]

# Find start and end of each trial from breath time
start_test_n, end_test_n = get_start_end_indices(data_normalized, ID)


In [12]:
def explory(data_dict, column):
    plt.figure(1)
    plt.plot(data_dict['data'][column])
    plt.title(column)
    plt.show()
    plt.figure(2)
    plt.hist(data_dict['data'][column])
    plt.title(column)
    plt.show()
    plt.figure(3)
    plt.boxplot(data_dict['data'][column])
    plt.title(column)
    plt.show()
    print(data_dict['data'][column].describe())
    print(data_dict['data'][column].isna().sum())